In [1]:
import geemap,ee
Map = geemap.Map(center=(39, 110), zoom=4)
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

### 矢量范围

In [ ]:
shp = r'F:\jupyter\gee\012极端事件\shp\面.shp'
shp_ee = geemap.shp_to_ee(shp,encoding='utf-8')
roi = shp_ee.geometry()


Map.addLayer(shp_ee, {}, 'range')

### 函数，去云、get_clip、ndvi

In [ ]:
# A function to mask out cloudy pixels.
def maskClouds(image):
  # Select the QA band.
  QA = image.select('StateQA')
  # Make a mask to get bit 10, the internal_cloud_algorithm_flag bit.
  bitMask = 1 << 10
  # Return an image masking out cloudy areas.
  return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))

In [ ]:
def get_cilp(roi):
    def cilp(image):
        return(image.clip(roi).copyProperties(image, ["system:time_start",'system:id']))
    return cilp

### ndvi计算

In [ ]:
#添加ndvi波段
def ndvi(img):
    ndvi = img.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')
    Msk = ndvi.lte(1).And(ndvi.gt(0))
    ndvi = ndvi.updateMask(Msk)
    return img.addBands(ndvi).copyProperties(img, ["system:time_start",'system:id'])

clip = get_cilp(roi)

In [ ]:
collection_09a1 = ee.ImageCollection("MODIS/061/MOD09A1")\
                                    .select(['sur_refl_b02','sur_refl_b01','StateQA'])\
                                    .filter(ee.Filter.bounds(roi))\
                                    .map(maskClouds)\
                                    .map(get_cilp(roi))

In [ ]:
collection_ndvi = collection_09a1.map(ndvi).select('NDVI')

In [ ]:
print('Metadata properties: ', collection_ndvi.first().propertyNames().getInfo())

In [ ]:
Map.addLayer(collection_ndvi, {}, 'ndvi')

### LST影像集

In [ ]:
def rename(image):
    return image.rename('LST').multiply(0.02).copyProperties(image, ["system:time_start",'system:id'])



    

In [ ]:
collection_LST = ee.ImageCollection("MODIS/061/MOD11A2")\
            .map(get_cilp(roi))\
            .select('LST_Day_1km').map(rename)

In [ ]:
Map.addLayer(collection_LST.map(lambda img: img.subtract(273)), {'min':-30,'max':40}, 'LST')

### 循环计算eca

In [ ]:
month = (4,5)

In [ ]:
LSTs_i = collection_LST.filter(ee.Filter.calendarRange(*month,'month'))
NDVIs_i = collection_ndvi.filter(ee.Filter.calendarRange(*month,'month'))

In [ ]:
ls = LSTs_i.aggregate_array('system:index').getInfo()

In [ ]:
i = ls[0]

In [ ]:
image_lst = LSTs_i.filter(ee.Filter.equals('system:index',rightValue=i))

In [ ]:
dir_lst = r'F:\jupyter\gee\012极端事件\tif\lst'
dir_ndvi = r'F:\jupyter\gee\012极端事件\tif\ndvi'

In [ ]:
for i in ls[70:]:
    path_lst = dir_lst + '\\' + f'{i}.tif'
    path_ndvi = dir_ndvi + '\\' + f'{i}.tif'
    image_lst = LSTs_i.filter(ee.Filter.equals('system:index',rightValue=i)).first()
    image_ndvi = NDVIs_i.filter(ee.Filter.equals('system:index',rightValue=i)).first()
    geemap.download_ee_image(image_lst.clip(roi), path_lst, region = roi, scale=1000,  crs='epsg:4326')
    geemap.download_ee_image(image_ndvi.clip(roi), path_ndvi, region = roi, scale=1000,  crs='epsg:4326')
    

In [ ]:
ls

In [ ]:
it = iter(range(1,ls.size().getInfo()+1))

In [ ]:
dic = {k:ls[k] for k in range(len(ls))}

In [ ]:
{}

In [ ]:
ee.Dictionary(dic)

In [ ]:
dir(ls)

In [ ]:
NDVIs_i